# Выбор локации для скважины

Заказчиком является добывающая компания «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.
Нам нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль и меньшие риски.


Шаги для выбора локации:

- В избранном регионе ищем месторождения, для каждого определяем значения признаков;
- Строим модель и оцениваем объём запасов;
- Выбираем месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy as np
from scipy.stats import t

In [2]:
data_r1 = pd.read_csv('geo_data_0.csv')
data_r2 = pd.read_csv('geo_data_1.csv')
data_r3 = pd.read_csv('geo_data_2.csv')

In [3]:
data_r1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_r1.id.duplicated().sum()

10

In [5]:
data_r1[data_r1.id.duplicated(keep=False) == True]

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
16633,fiKDv,0.157341,1.028359,5.585586,95.817889
21426,Tdehs,0.829407,0.298807,-0.049563,96.035308
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205


В данных по первому региону найдено 10 дубликатов по `id`, удалим их.

In [6]:
data_r1 = data_r1.drop_duplicates(subset=['id'])

In [7]:
data_r1 = data_r1.reset_index(drop=True)

In [8]:
data_r1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99990 entries, 0 to 99989
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99990 non-null  object 
 1   f0       99990 non-null  float64
 2   f1       99990 non-null  float64
 3   f2       99990 non-null  float64
 4   product  99990 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Готово.

In [9]:
data_r2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [10]:
data_r2.id.duplicated().sum()

4

In [11]:
data_r2[data_r2.id.duplicated(keep=False) == True]

,id,f0,f1,f2,product
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


Аналогично удалим 4 дубликата по второму региону.

In [12]:
data_r2 = data_r2.drop_duplicates(subset=['id'])

In [13]:
data_r2 = data_r2.reset_index(drop=True)

In [14]:
data_r2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99996 entries, 0 to 99995
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99996 non-null  object 
 1   f0       99996 non-null  float64
 2   f1       99996 non-null  float64
 3   f2       99996 non-null  float64
 4   product  99996 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [15]:
data_r3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [16]:
data_r3.id.duplicated().sum()

4

In [17]:
data_r3[data_r3.id.duplicated(keep=False) == True]

,id,f0,f1,f2,product
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


И в данных третьего региона удалим 4 дубликата.

In [18]:
data_r3 = data_r3.drop_duplicates(subset=['id'])

In [19]:
data_r3 = data_r3.reset_index(drop=True)

In [20]:
data_r3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99996 entries, 0 to 99995
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99996 non-null  object 
 1   f0       99996 non-null  float64
 2   f1       99996 non-null  float64
 3   f2       99996 non-null  float64
 4   product  99996 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Данные обработаны и готовы для обучения моделей.

## Обучение и проверка модели

In [21]:
def training_model(data):
    new_data = data.drop('id', axis=1)
    features = new_data.drop('product', axis=1)
    target = new_data['product']
    features_train, features_valid, target_train, target_valid =  train_test_split(features, target, test_size=0.25, random_state=222)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    scores = abs(cross_val_score(estimator=model, X=features_valid, y=target_valid, cv=5, scoring='neg_root_mean_squared_error'))
    final_score = sum(scores) / len(scores)
    return predictions, final_score, features_train, features_valid, target_train, target_valid

In [22]:
result_r1 = training_model(data_r1)
print(f'RMSE региона 1: {result_r1[1]}')   
print(f'Средний предсказанный запас сырья, тыс. бар: {result_r1[0].mean()}')

RMSE региона 1: 37.686724030042114
Средний предсказанный запас сырья, тыс. бар: 92.59425562354359


In [23]:
result_r2 = training_model(data_r2)
print(f'RMSE региона 2: {result_r2[1]}')   
print(f'Средний предсказанный запас сырья, тыс. бар: {result_r2[0].mean()}')

RMSE региона 2: 0.8863678535714312
Средний предсказанный запас сырья, тыс. бар: 68.69896994322433


In [24]:
result_r3 = training_model(data_r3)
print(f'RMSE региона 3: {result_r3[1]}')   
print(f'Средний предсказанный запас сырья, тыс. бар.: {result_r3[0].mean()}')

RMSE региона 3: 40.04043744045968
Средний предсказанный запас сырья, тыс. бар.: 95.05059715461914


Можно заметить, что модель очень точно спрогнозировала значения во втором регионе относительно остальных двух.

## Подготовка к расчёту прибыли

In [25]:
PROFIT = 450000
BUDGET = 10_000_000_000
min_profit = BUDGET / 200
min_volume = min_profit / PROFIT

In [26]:
print(f'Достаточный объём сырья для безубыточной разработки новой скважины: {min_volume} тыс. бар.')

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111 тыс. бар.


Значение достаточного объёма сырья для безубыточной разработки новой скважины выше средних предсказанных показателей. Этого стоило ожидать, так как некоторые скважины могут добывать совсем немного нефти, когда другие могут и превышать этот порог в несколько раз, именно такие скважины нам придётся и отыскать.

## Расчёт прибыли и рисков 

In [27]:
def revenue(prediction, target, count):
    target = target.reset_index(drop=True)
    pred_sort = pd.Series(prediction).sort_values(ascending=False)
    selected = target[pred_sort.index][:count]
    volume = selected.sum()
    full_profit = ((volume * PROFIT) / 1_000_000_000) - 10
    return volume, full_profit

In [28]:
top_200_r1_volume = revenue(training_model(data_r1)[0], training_model(data_r1)[5], 200)[0]
top_200_r1_full_profit = revenue(training_model(data_r1)[0], training_model(data_r1)[5], 200)[1]

In [29]:
print(f'Сумма объёма сырья лучших 200 скважин 1-ого региона, если верить предсказаниям: {top_200_r1_volume} тыс. бар.')
print(f'Прибыль для полученного объёма сырья 1-ого региона, если верить предсказаниям: {top_200_r1_full_profit} млрд рублей')

Сумма объёма сырья лучших 200 скважин 1-ого региона, если верить предсказаниям: 30037.984304537407 тыс. бар.
Прибыль для полученного объёма сырья 1-ого региона, если верить предсказаниям: 3.5170929370418342 млрд рублей


In [30]:
top_200_r2_volume = revenue(training_model(data_r2)[0], training_model(data_r2)[5], 200)[0]
top_200_r2_full_profit = revenue(training_model(data_r2)[0], training_model(data_r2)[5], 200)[1]

In [31]:
print(f'Сумма объёма сырья лучших 200 скважин 2-ого региона, если верить предсказаниям: {top_200_r2_volume} тыс. бар.')
print(f'Прибыль для полученного объёма сырья 2-ого региона, если верить предсказаниям: {top_200_r2_full_profit} млрд рублей')

Сумма объёма сырья лучших 200 скважин 2-ого региона, если верить предсказаниям: 27589.081548181137 тыс. бар.
Прибыль для полученного объёма сырья 2-ого региона, если верить предсказаниям: 2.415086696681511 млрд рублей


In [32]:
top_200_r3_volume = revenue(training_model(data_r3)[0], training_model(data_r3)[5], 200)[0]
top_200_r3_full_profit = revenue(training_model(data_r3)[0], training_model(data_r3)[5], 200)[1]

In [33]:
print(f'Сумма объёма сырья лучших 200 скважин 3-ого региона, если верить предсказаниям: {top_200_r3_volume} тыс. бар.')
print(f'Прибыль для полученного объёма сырья 3-ого региона, если верить предсказаниям: {top_200_r3_full_profit} млрд рублей')

Сумма объёма сырья лучших 200 скважин 3-ого региона, если верить предсказаниям: 27575.106543037917 тыс. бар.
Прибыль для полученного объёма сырья 3-ого региона, если верить предсказаниям: 2.408797944367061 млрд рублей


In [34]:
def bootstrap(target, predictions):
    values = []
    state = np.random.RandomState(222)
    for i in range(1000):
        target = target.reset_index(drop=True)
        target_subsample = target.sample(n=500, random_state=state, replace=True)
        predictions_subsample = predictions[target_subsample.index]
        values.append(revenue(predictions_subsample, target_subsample, 200)[1])
    value = pd.Series(values)
    mean_profit = value.mean()
    max_profit = value.max()
    min_profit = value.min()
    risk = sum(value<0)*100 / len(value)
    conf_interval = t.interval(0.95, len(value)-1, value.mean(), value.sem())
    lower = value.quantile(0.025)
    higher = value.quantile(0.975)
    return value, mean_profit, max_profit, min_profit, risk, conf_interval, lower, higher

In [35]:
final_r1 = (bootstrap(training_model(data_r1)[5], training_model(data_r1)[0]))

In [36]:
print('Данные для 1-ого региона:\n')
print(f'Средняя прибыль: {final_r1[1]:.3} млрд. рублей')
print(f'Максимальная прибыль: {final_r1[2]:.3} млрд. рублей')
print(f'Минимальная прибыль: {final_r1[3]:.3} млрд. рублей')
print(f'Риск убытка: {final_r1[4]} %')
print(f'Доверительный интервал: {final_r1[5]} млрд. рублей')
print(f'2,5%-квантиль: {final_r1[6]:.3} млрд. рублей')
print(f'97,5%-квантиль: {final_r1[7]:.3} млрд. рублей')

Данные для 1-ого региона:

Средняя прибыль: 0.449 млрд. рублей
Максимальная прибыль: 1.23 млрд. рублей
Минимальная прибыль: -0.234 млрд. рублей
Риск убытка: 5.2 %
Доверительный интервал: (0.43249848866213275, 0.4656134148338789) млрд. рублей
2,5%-квантиль: -0.0654 млрд. рублей
97,5%-квантиль: 0.942 млрд. рублей


In [37]:
final_r2 = (bootstrap(training_model(data_r2)[5], training_model(data_r2)[0]))

In [38]:
print('Данные для 2-ого региона:\n')
print(f'Средняя прибыль: {final_r2[1]:.3} млрд. рублей')
print(f'Максимальная прибыль: {final_r2[2]:.3} млрд. рублей')
print(f'Минимальная прибыль: {final_r2[3]:.3} млрд. рублей')
print(f'Риск убытка: {final_r2[4]} %')
print(f'Доверительный интервал: {final_r2[5]} млрд. рублей')
print(f'2,5%-квантиль: {final_r2[6]:.3} млрд. рублей')
print(f'97,5%-квантиль: {final_r2[7]:.3} млрд. рублей')

Данные для 2-ого региона:

Средняя прибыль: 0.442 млрд. рублей
Максимальная прибыль: 1.07 млрд. рублей
Минимальная прибыль: -0.246 млрд. рублей
Риск убытка: 1.9 %
Доверительный интервал: (0.4296524900610265, 0.4550084658909382) млрд. рублей
2,5%-квантиль: 0.0288 млрд. рублей
97,5%-квантиль: 0.84 млрд. рублей


In [39]:
final_r3 = (bootstrap(training_model(data_r3)[5], training_model(data_r3)[0]))

In [40]:
print('Данные для 3-ого региона:\n')
print(f'Средняя прибыль: {final_r3[1]:.3} млрд. рублей')
print(f'Максимальная прибыль: {final_r3[2]:.3} млрд. рублей')
print(f'Минимальная прибыль: {final_r3[3]:.3} млрд. рублей')
print(f'Риск убытка: {final_r3[4]} %')
print(f'Доверительный интервал: {final_r3[5]} млрд. рублей')
print(f'2,5%-квантиль: {final_r3[6]:.3} млрд. рублей')
print(f'97,5%-квантиль: {final_r3[7]:.3} млрд. рублей')

Данные для 3-ого региона:

Средняя прибыль: 0.375 млрд. рублей
Максимальная прибыль: 1.24 млрд. рублей
Минимальная прибыль: -0.51 млрд. рублей
Риск убытка: 7.0 %
Доверительный интервал: (0.3591537087580651, 0.39129777604485333) млрд. рублей
2,5%-квантиль: -0.138 млрд. рублей
97,5%-квантиль: 0.885 млрд. рублей


Только один регион имеет риск убытка менее 2,5 %, это регион под номером 2.

Исследование можно считать успешным, так как по его итогам для добывающей компании «ГлавРосГосНефть» был определён регион, где добыча принесёт наибольшую прибыль. Этим регионом оказался "Регион-2", риски убытка составили 1,9 %, а потенциальная средняя прибыль - 442 миллиона рублей, что составляет ~4.4 % от изначального бюджета.